# Hồi quy đa biến Multiple Regression (Diễn giải)

Trong notebook này, chúng ta sẽ sử dụng dữ liệu doanh số bán nhà ở Quận King để dự đoán giá nhà sử dụng hồi quy tuyến tính đa biến. Chúng ta sẽ:
* Thực hiện một số thiết kế đặc trưng bằng các hàm DataFrame nội bộ.
* Sử dụng các hàm sklearn có sẵn để tính hồi quy và truy cập các tham số của nó (hệ số).
* Viết hàm tính RSS với các trọng số hồi quy, yếu tố dự báo và đầu ra đã cho.
* Xem các hệ số và diễn giải ý nghĩa của chúng.
* Đánh giá mô hình đa biến qua RSS.

## Import thư viện

In [1]:
import pandas as pd
import numpy as np

## Load dữ liệu bán nhà

Tập dữ liệu từ doanh số bán nhà quận King, Seatle, WA.

In [2]:
full_data = pd.read_csv("kc_house_data.csv", index_col=0)
full_data.head()

,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
id,,,,,,,,,,,,,,,,,,,,
7129300520,20141013T000000,221900.0,3,1.00,1180,5650,1.0,0,0,3,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650
6414100192,20141209T000000,538000.0,3,2.25,2570,7242,2.0,0,0,3,7,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639
5631500400,20150225T000000,180000.0,2,1.00,770,10000,1.0,0,0,3,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062
2487200875,20141209T000000,604000.0,4,3.00,1960,5000,1.0,0,0,5,7,1050,910,1965,0,98136,47.5208,-122.393,1360,5000
1954400510,20150218T000000,510000.0,3,2.00,1680,8080,1.0,0,0,3,8,1680,0,1987,0,98074,47.6168,-122.045,1800,7503


## Chia dữ liệu thành tập huấn luyện và kiểm tra.
Chúng ta sử dụng seed=0 để mọi người chạy notebook này có đều được kết quả tương tự. Thực tế, các bạn có thể thiết lập seed ngẫu nhiên để đảm bảo tính nhất quán. 

In [3]:
from sklearn.model_selection import train_test_split
train_data, test_data = train_test_split(full_data, train_size=0.8, test_size=0.2, random_state=0)
train_data = train_data.copy()
test_data = test_data.copy()

# Tìm hiểu mô hình hồi quy đa biến

Chúng ta có thể sử dụng code sau để tìm hiểu mô hình hồi quy đa biến dự đoán 'price' dựa trên các đặc trưng sau: example_features = ['sqft_living', 'bedrooms', 'bathrooms'] trong dữ liệu huấn luyện với code:

In [4]:
def extract_features(data, features_title):
    # nhận các đặc trưng từ DataFrame
    features = [data[title].values for title in features_title]
    # xếp chồng chúng thành 2d [examples, properties]
    return np.stack(features, axis=-1)

In [5]:
from sklearn.linear_model import LinearRegression

example_features_title = ['sqft_living', 'bedrooms', 'bathrooms']
# trích xuất đặc trưng thành các mảng numpy và xếp chồng chúng mỗi example 
example_features = extract_features(train_data, example_features_title)
example_labels = train_data['price']
example_model = LinearRegression().fit(example_features, example_labels)

Chúng ta đã khớp mô hình, giờ có thể trích xuất các trọng số mô hình (hệ số) từ mô hình như sau:

In [6]:
example_weight_summary = example_model.coef_
print(example_weight_summary)

[   313.17055038 -56754.66651422   6887.71910816]


## Đưa ra dự đoán

Trong notebook về gradient descent, chúng ta sử dụng numpy để thực hiện hồi quy. Trong notebook này, chúng ta sẽ sử dụng các hàm giao diện của mô hình sklearn hiện có để phân tích hồi quy đa biến.

Khi mô hình đã xây, chúng ta có thể sử dụng hàm `predict` để tìm các giá trị dự đoán cho dữ liệu mà chúng ta truyền vào. Ví dụ: sử dụng example_model trên: 

In [7]:
example_predictions = example_model.predict(example_features)
print(example_predictions[0]) # should be 395813.499

395813.4988028944


## Tính RSS

Bây giờ chúng ta có thể đưa ra các dự đoán cho mô hình, hãy viết một hàm tính RSS của mô hình. Hoàn thành hàm dưới đây để tính RSS với mô hình, dữ liệu và đầu ra đã biết.

In [11]:
def get_residual_sum_of_squares(model, data, outcome):
    # Trước tiên lấy các dự đoán
    y_pr = model.predict(data)
    # Sau đó tính các phần dư/lỗi
    
    RSS = np.sum((outcome - y_pr)**2)

    return(RSS)    

Kiểm tra hàm bằng cách tính RSS trong dữ liệu KIỂM TRA cho mô hình mẫu:

In [13]:
example_test_features = extract_features(test_data, example_features_title)
example_test_labels = test_data['price']
rss_example_test = get_residual_sum_of_squares(example_model, example_test_features, example_test_labels)
print(rss_example_test) # should be ~ 2.5921e+14

259213572106085.25


# Tạo một số đặc trưng mới

Chúng ta thường nghĩ hồi quy đa biến gồm nhiều đặc trưng khác nhau (ví dụ: số phòng ngủ, diện tích và số phòng tắm), nhưng chúng ta cũng có thể xem xét việc biến đổi các đối đặc trưng hiện có, ví dụ: log của squarefeet hoặc thậm chí các đặc trưng "tương tác" như tích của số phòng ngủ và số phòng tắm.

Giờ chúng ta sẽ sử dụng hàm logarit mặc định của python để tạo đặc trưng mới. Chúng ta cần import nó từ thư viện math.

In [21]:
from math import log

Tiếp theo, chúng ta sẽ tạo 4 đặc trưng mới sau làm cột trong cả dữ liệu HUẤN LUYỆN và KIỂM TRA:
* bedrooms_squared = bedrooms\*bedrooms
* bed_bath_rooms = bedrooms\*bathrooms
* log_sqft_living = log(sqft_living)
* lat_plus_long = lat + long 

Ví dụ như sau:

In [22]:
train_data['bedrooms_squared'] = train_data['bedrooms'].map(lambda x: x**2)
test_data['bedrooms_squared'] = test_data['bedrooms'].map(lambda x: x**2)

In [24]:
# tạo 3 đặc trưng còn lại trong cả dữ liệu HUẤN LUYỆN và KIỂM TRA
train_data['bed_bath_rooms'] = train_data['bedrooms'] * train_data['bathrooms']
test_data['bed_bath_rooms'] = test_data['bedrooms'] * test_data['bathrooms']

train_data['log_sqft_living'] = train_data['sqft_living'].map(lambda x : log(x))
test_data['log_sqft_living'] = test_data['sqft_living'].map(lambda x : log(x))

train_data['lat_plus_long'] = train_data['lat'] + train_data['long']
test_data['lat_plus_long'] = test_data['lat'] + test_data['long']

* bedrooms*bedrooms sẽ tăng phân tách giữa ít phòng ngủ (chẳng hạn: 1) và nhiều phòng ngủ (chẳng hạn: 4). Do đó, đặc trưng này phần lớn sẽ ảnh hưởng với nhiều phòng ngủ.
* bedrooms*bathrooms cho đặc trưng "tương tác", nó sẽ lớn khi cả hai đều lớn.
* log(sqft_living) khiến các giá trị lớn hơn gần nhau hơn và lan ra các giá trị nhỏ.
* lat + long hoàn toàn không có ý nghĩa nhưng chúng ta vẫn thực hiện (bạn sẽ biết lý do sau).

**Quiz: Giá trị trung bình cộng của 4 đặc trưng mới trong dữ liệu KIỂM TRA là bao nhiêu? (làm tròn tới 2 chữ số thập phân)**

In [27]:
# pandas.Series có hàm gọi chính xác như vậy.
bebs = pd.Series.mean(test_data['bedrooms_squared'])
bebs_bath = pd.Series.mean(test_data['bed_bath_rooms'])
sqft_liv = pd.Series.mean(test_data['log_sqft_living'])
lat_long = pd.Series.mean(test_data['lat_plus_long'])
bebs,bebs_bath,sqft_liv,lat_long

(12.210501966227158, 7.447721489706223, 7.550239467794731, -74.65426069858911)

## Tìm hiểu mô hình đa biến

Bây giờ chúng ta sẽ tìm hiểu trọng số của ba mô hình (lồng nhau) dự đoán giá nhà. Mô hình đầu tiên sẽ ít đặc trưng nhất, mô hình thứ hai sẽ thêm một đặc trưng và mô hình thứ ba sẽ thêm vài đặc trưng khác:
* Mô hình 1: sqft_living, # bedrooms, # bathrooms, lat & long
* Mô hình 2: thêm bedrooms\*bathrooms
* Mô hình 3: thêm log_sqft, bedrooms_squared, và lat_plus_long

In [37]:
model_1_features = ['sqft_living', 'bedrooms', 'bathrooms', 'lat', 'long']
model_2_features = model_1_features + ['bed_bath_rooms']
model_3_features = model_2_features + ['bedrooms_squared', 'log_sqft_living', 'lat_plus_long']

Bạn đã có các đặc trưng, trọng số cho 3 mô hình khác nhau dự đoán target = 'price' sử dụng LinearRegression của sklearn và thấy giá trị trọng số/hệ số:

*Lưu ý: mô hình hồi quy trong khóa này thường có `coef_`*

In [38]:
# Tìm hiểu 3 mô hình. Trích xuất đặc trưng và khớp mô hình với các đặc trưng tương ứng. 
# Nếu không nhớ, hãy xem example_model bên trên
example_features1 = extract_features(train_data,model_1_features)
example_labels = train_data['price']
example_model1 = LinearRegression().fit(example_features1, example_labels)
example_model1.coef_

array([ 3.12942010e+02, -5.30962691e+04,  1.47770428e+04,  6.53983343e+05,
       -3.25707336e+05])

In [39]:
example_features2 = extract_features(train_data,model_2_features)
example_labels = train_data['price']
example_model2 = LinearRegression().fit(example_features2, example_labels)
example_model2.coef_

array([ 3.06819573e+02, -1.04604718e+05, -7.01815289e+04,  6.50590952e+05,
       -3.09965751e+05,  2.49441497e+04])

In [40]:
example_features3 = extract_features(train_data,model_3_features)
example_labels = train_data['price']
example_model3 = LinearRegression().fit(example_features3, example_labels)
example_model3.coef_

array([ 5.37808086e+02,  2.78047852e+03,  1.01363766e+05,  1.74471440e+12,
        1.74471346e+12, -1.81822551e+04,  7.24579933e+02, -5.71030023e+05,
       -1.74471375e+12])

**Quiz: Dấu (dương hoặc âm) của hệ số/trọng số cho 'bathrooms' trong mô hình 1 là gì?**

**Quiz: Dấu (dương hoặc âm) của hệ số/trọng số cho 'bathrooms' trong mô hình 2 là gì?**

Hãy nghĩ xem điều này có ý nghĩa gì.

## So sánh các mô hình đa biến

Chúng ta đã nghiên cứu 3 mô hình và trích xuất trong số mô hình mà chúng ta muốn đánh giá là tốt nhất.

Trước tiên sử dụng các hàm trước đó để tính RSS trong dữ liệu HUẤN LUYỆN cho từng mô hình.

In [41]:
# Tính RSS trong dữ liệu HUẤN LUYỆN cho từng mô hình và hiển thị các giá trị. 
# Xem lab trước nếu quên cách làm.
rss_example_train1 = get_residual_sum_of_squares(example_model1, example_features1, example_labels)
rss_example_train1

489921798794164.5

In [42]:
rss_example_train2 = get_residual_sum_of_squares(example_model2, example_features2, example_labels)
rss_example_train2

485399599864788.94

In [43]:
rss_example_train3= get_residual_sum_of_squares(example_model3, example_features3, example_labels)
rss_example_train3

456826822222784.9

**Quiz: Mô hình nào (1, 2 hay 3) có RSSS thấp nhất trong dữ liệu HUẤN LUYỆN?** Đây có phải điều chúng ta dự kiến?

Bây giờ hãy tính RSS trong dữ liệu KIỂM TRA cho từng mô hình. 

In [45]:
# Tính RSS trong dữ liệu KIỂM TRA cho từng mô hình và hiển thị các giá trị.
example_test_features1 = extract_features(test_data, model_1_features)
example_test_labels = test_data['price']
rss_example_test1 = get_residual_sum_of_squares(example_model1, example_test_features1, example_test_labels)
print(rss_example_test)

129606786053042.62


In [47]:
# Tính RSS trong dữ liệu KIỂM TRA cho từng mô hình và hiển thị các giá trị.
example_test_features2 = extract_features(test_data, model_2_features)
example_test_labels = test_data['price']
rss_example_test2 = get_residual_sum_of_squares(example_model2, example_test_features2, example_test_labels)
print(rss_example_test)

129606786053042.62


In [48]:
# Tính RSS trong dữ liệu KIỂM TRA cho từng mô hình và hiển thị các giá trị.
example_test_features3 = extract_features(test_data, model_3_features)
example_test_labels = test_data['price']
rss_example_test3 = get_residual_sum_of_squares(example_model3, example_test_features3, example_test_labels)
print(rss_example_test)

129606786053042.62


**Quiz: Mô hình nào (1, 2 hay 3) có RSSS thấp nhất trong dữ liệu KIỂM TRA?** Đây có phải điều chúng ta dự kiến? Nghĩ về các đặc trưng đã thêm vào từng mô hình trước đó. 